In [15]:
import pyomo.environ as pyEnv

#Custo de transporte da fábrica i para o cliente j
cij = [[162, 247],
       [117, 193],
       [131, 185]]

capacidades = [1000, 1500, 1200]
demandas = [2300, 1400]

m = len (capacidades)
n = len (demandas)

# Declaraçao do Modelo e seus Parâmetros
# Modelo
modelo = pyEnv.ConcreteModel()

# Índice para as fábricas
modelo.I = pyEnv.RangeSet(m)

# Índice para os clientes
modelo.J = pyEnv.RangeSet(n)

# Variáveis de decisão xij
modelo.x = pyEnv.Var(modelo.I, modelo.J, within = pyEnv.NonNegativeReals)

# Custo de transporte da fábrica i para o cliente j
modelo.c = pyEnv.Param(modelo.I, modelo.J, initialize = lambda modelo, i, j: cij[i-1][j-1])

# Capacidade de cada fábrica
modelo.b = pyEnv.Param(modelo.I, initialize = lambda modelo, i: capacidades[i-1])

# Demanda de cada cliente
modelo.d = pyEnv.Param(modelo.J, initialize = lambda modelo, j: demandas [j-1])

# Declaração da Função Objetivo e Restrições
def func_objetivo(modelo):
    return sum(modelo.x[i,j] * modelo.c[i,j] for i in modelo.I for j in modelo.J)

modelo.objetivo = pyEnv.Objective (rule = func_objetivo, sense = pyEnv.minimize)

# RESTRIÇÃO 1 - Cada Fábrica não exceder sua capacidade
def rule_rest1 (modelo, i):
    return sum (modelo.x[i,j] for j in modelo.J) <= modelo.b[i]

modelo_rest1 = pyEnv.Constraint(modelo.I, rule = rule_rest1)

# RESTRIÇÃO 2 - Cada cliente ter sua demanda atendida
def rule_rest2(modelo, j):
    return sum (modelo.x[i,j] for i in modelo.I) >= modelo.d[j]

modelo.rest2 = pyEnv.Constraint(modelo.J, rule = rule_rest2)

# RESOLULÃO DO MODELO
resultado = pyEnv.SolverFactory('glpk').solve(modelo)
modelo.objetivo()
print(resultado)

# Print das variáveis de decisão escolhidas
lista = list(modelo.x.keys())
for i in lista:
    if modelo.x[i]() != 0:
       print (i, '--', modelo.x[i]())





Problem: 
- Name: unknown
  Lower bound: 528100.0
  Upper bound: 528100.0
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 6
  Number of nonzeros: 6
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.7416813373565674
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

(2, 1) -- 2300.0
(3, 2) -- 1400.0
